In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
import toml
config = toml.load('../../../configuration/input_configuration.toml')

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'

# Jobs Accessible within 45 Minutes of Transit

In [6]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(r'..\..\..\outputs\access\transit_jobs_access.csv'); incr+=1
df_lu = pd.read_csv(r'..\..\..\outputs\landuse\parcels_urbansim.txt',
                   sep='\s+', usecols=['parcelid','emptot_p'])
tot_jobs = df_lu['emptot_p'].sum()

### Region

In [7]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_region.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Jobs within 45-minute Transit Commute,% Total Jobs
"137,507",6.3%


### County

In [8]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_county.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"232,791",10.6%
Kitsap,"6,351",0.3%
Outside Region,0,0.0%
Pierce,"19,341",0.9%
Snohomish,"26,210",1.2%


### Regional Growth Center

In [9]:
_df = df[df['geography_group'] == 'rgc_binary']
_df.index = _df.geography.map({'0': 'Not in RGC', '1': 'All RGCs'})
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Not in RGC,"101,906",4.7%
All RGCs,"472,786",21.6%


In [10]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Auburn,"66,679",3.0%
Bellevue,"461,114",21.0%
Bothell Canyon Park,"39,160",1.8%
Bremerton,"34,597",1.6%
Burien,"122,503",5.6%
Everett,"57,765",2.6%
Federal Way,"98,342",4.5%
Greater Downtown Kirkland,"254,334",11.6%
Issaquah,0,0.0%


### Regional Geography

In [11]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_reg_geog.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
CitiesTowns,"10,549",0.5%
Core,"59,440",2.7%
HCT,"37,439",1.7%
Metro,"338,828",15.5%
UU,"10,157",0.5%


### Equity Geographies

In [12]:
geo_list = []
equity_geogs = ['youth','elderly','english','racial','poverty','disability']
for equity_geog in equity_geogs:
    for geog_type in ['_geog_vs_reg_total','_geog_vs_50_percent']:
        if geog_type == '_geog_vs_reg_total' or equity_geog in ['poverty','racial']:
            geo_list.append(equity_geog+geog_type)

In [13]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
youth_geog_vs_reg_total,"46,991",2.1%
elderly_geog_vs_reg_total,"97,814",4.5%
english_geog_vs_reg_total,"124,116",5.7%
racial_geog_vs_reg_total,"156,959",7.2%
racial_geog_vs_50_percent,"167,157",7.6%
poverty_geog_vs_reg_total,"135,562",6.2%
poverty_geog_vs_50_percent,"310,413",14.2%
disability_geog_vs_reg_total,"112,620",5.1%


In [14]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
NOT in youth_geog_vs_reg_total,"211,712",9.7%
NOT in elderly_geog_vs_reg_total,"173,414",7.9%
NOT in english_geog_vs_reg_total,"145,396",6.6%
NOT in racial_geog_vs_reg_total,"121,121",5.5%
NOT in racial_geog_vs_50_percent,"124,068",5.7%
NOT in poverty_geog_vs_reg_total,"138,807",6.3%
NOT in poverty_geog_vs_50_percent,"133,730",6.1%
NOT in disability_geog_vs_reg_total,"158,428",7.2%


# Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.
Average accessible jobs are weighted averages based on parcel household population.

In [15]:
df = pd.read_csv(r'..\..\..\outputs\access\walk_bike_jobs_access.csv')

### Regional Average

In [16]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
"11,172","54,354",0.5%,2.5%


### Home County

In [17]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_county.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
King,"18,519","84,915",0.8%,3.9%
Kitsap,"1,159","7,422",0.1%,0.3%
Outside Region,0,31,0.0%,0.0%
Pierce,"2,537","18,572",0.1%,0.8%
Snohomish,"2,036","17,994",0.1%,0.8%


### Regional Growth Center

In [18]:
_df = df[df['geography_group'] == 'rgc_binary']
_df.index = _df.geography_value.map({'0': 'Not in RGC', '1': 'All RGCs'})
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
Not in RGC,"2,923","34,998",0.1%,1.6%
All RGCs,"88,863","236,642",4.1%,10.8%


In [19]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
Auburn,"10,541","40,324",0.5%,1.8%
Bellevue,"59,188","110,699",2.7%,5.1%
Bothell Canyon Park,"8,539","21,748",0.4%,1.0%
Bremerton,"11,478","29,603",0.5%,1.4%
Burien,"4,829","13,400",0.2%,0.6%
Everett,"15,192","35,065",0.7%,1.6%
Federal Way,"6,348","26,221",0.3%,1.2%
Greater Downtown Kirkland,"12,208","36,559",0.6%,1.7%
Issaquah,NaN,NaN,NaN,NaN


### Regional Geography

In [20]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_reg_geog.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
CitiesTowns,984,"8,948",0.0%,0.4%
Core,"3,416","28,646",0.2%,1.3%
HCT,"1,495","15,923",0.1%,0.7%
Metro,"29,414","127,845",1.3%,5.8%
UU,443,"7,024",0.0%,0.3%


### Equity Geographies

In [21]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
youth_geog_vs_reg_total,"1,735","19,322",0.1%,0.9%
elderly_geog_vs_reg_total,"8,544","40,333",0.4%,1.8%
english_geog_vs_reg_total,"10,839","48,404",0.5%,2.2%
racial_geog_vs_reg_total,"18,063","69,071",0.8%,3.2%
racial_geog_vs_50_percent,"18,653","70,871",0.9%,3.2%
poverty_geog_vs_reg_total,"13,811","57,047",0.6%,2.6%
poverty_geog_vs_50_percent,"30,974","101,410",1.4%,4.6%
disability_geog_vs_reg_total,"12,956","51,240",0.6%,2.3%


In [22]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
NOT in youth_geog_vs_reg_total,"18,909","83,074",0.9%,3.8%
NOT in elderly_geog_vs_reg_total,"13,551","67,038",0.6%,3.1%
NOT in english_geog_vs_reg_total,"11,369","57,859",0.5%,2.6%
NOT in racial_geog_vs_reg_total,"5,368","41,956",0.2%,1.9%
NOT in racial_geog_vs_50_percent,"7,782","46,868",0.4%,2.1%
NOT in poverty_geog_vs_reg_total,"9,408","52,553",0.4%,2.4%
NOT in poverty_geog_vs_50_percent,"10,740","53,326",0.5%,2.4%
NOT in disability_geog_vs_reg_total,"9,673","56,971",0.4%,2.6%


# Intersection Density

In [23]:
df = pd.read_csv(r'..\..\..\outputs\landuse\buffered_parcels.txt', delim_whitespace=True)

# Total intersections within 1/2 mile buffer
df['intersections'] = df[['nodes3_2','nodes4_2']].sum(axis=1)
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../inputs/db/'+config['db_name'])
df = df.merge(parcel_geog, left_on='parcelid', right_on='ParcelID')

### County

In [24]:
# population weighted measure
df['intersections_wt'] = df['intersections']*df['hh_p']
_df = pd.DataFrame(df.groupby('CountyName').sum()['intersections_wt']/df.groupby('CountyName').sum()['hh_p'])
_df.columns = ['Intersections']
_df

,Intersections
CountyName,
King,155
Kitsap,51
Outside Region,4
Pierce,80
Snohomish,73


### Regional Growth Center

In [25]:
# population weighted measure
df['intersections_wt'] = df['intersections']*df['hh_p']
_df = pd.DataFrame(df.groupby('GrowthCenterName').sum()['intersections_wt']/df.groupby('GrowthCenterName').sum()['hh_p'])
_df.columns = ['Intersections']
_df

,Intersections
GrowthCenterName,
Auburn,192
Bellevue,273
Bothell Canyon Park,69
Bremerton,168
Burien,176
Everett,161
Federal Way,134
Greater Downtown Kirkland,167
Issaquah,NaN


### Regional Geography

In [26]:
# population weighted measure
df['intersections_wt'] = df['intersections']*df['hh_p']
_df = pd.DataFrame(df.groupby('GrowthCenterName').sum()['intersections_wt']/df.groupby('GrowthCenterName').sum()['hh_p'])
_df.columns = ['Intersections']
_df

,Intersections
GrowthCenterName,
Auburn,192
Bellevue,273
Bothell Canyon Park,69
Bremerton,168
Burien,176
Everett,161
Federal Way,134
Greater Downtown Kirkland,167
Issaquah,NaN


### Equity Geography

In [27]:
col = 'youth_geog_vs_reg_total'
df['intersections_wt'] = df['intersections']*df['hh_p']
_df = pd.DataFrame(df.groupby(col).sum()['intersections_wt']/df.groupby(col).sum()['hh_p'])

In [28]:
# population weighted measure
results_df = pd.DataFrame()
for col in geo_list:
    df['intersections_wt'] = df['intersections']*df['hh_p']
    _df = pd.DataFrame(df.groupby(col).sum()['intersections_wt']/df.groupby(col).sum()['hh_p'])
    try:
        results_df[col] = _df.loc[1]
    except:
        next
results_df = results_df.T
results_df.columns = ['Intersections']
results_df

,Intersections
youth_geog_vs_reg_total,78
elderly_geog_vs_reg_total,101
english_geog_vs_reg_total,120
racial_geog_vs_reg_total,136
racial_geog_vs_50_percent,140
poverty_geog_vs_reg_total,127
poverty_geog_vs_50_percent,198
disability_geog_vs_reg_total,115


In [29]:
# population weighted measure
results_df = pd.DataFrame()
for col in geo_list:
    df['intersections_wt'] = df['intersections']*df['hh_p']
    _df = pd.DataFrame(df.groupby(col).sum()['intersections_wt']/df.groupby(col).sum()['hh_p'])
    results_df[col] = _df.loc[0]
results_df = results_df.T
results_df.columns = ['Intersections']
results_df.index = ['NOT in '+ i for i in results_df.index]
results_df

,Intersections
NOT in youth_geog_vs_reg_total,152
NOT in elderly_geog_vs_reg_total,134
NOT in english_geog_vs_reg_total,117
NOT in racial_geog_vs_reg_total,103
NOT in racial_geog_vs_50_percent,109
NOT in poverty_geog_vs_reg_total,112
NOT in poverty_geog_vs_50_percent,117
NOT in disability_geog_vs_reg_total,121
